<a id="top"></a>
# Visualization of Roman APT products
***

## Server Information
IMPORTANT: To run this tutorial, please make sure you are logged in the RRN with a medium or large server. Running the parallelized examples in the advanced use cases will require a large server.


## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## Imports

- *numpy* to handle array functions
- *matplotlib* for plotting data
- *astropy.table* for creating tidy tables of the data
- *pandas* for creating tidy tables of the data
- *pysiaf* to get the coordinates of WFI in different frames of reference (used internally in `footprint_utils`)
- *healpy* to generate all-sky maps (used internally in `footprint_utils`)
- *hpgeom* to handle quick healpix operations (used internally in `footprint_utils`)
- *healsparse* to generate lightweight partial-sky high-resolution HEALPix maps
- *skyproj* to generate sky plots using different projections
- *dask* (Optional) to perform parallel processing

## Introduction

This notebook tries to illustrate several examples of how to visualize APT products in Python. Note that the APT GUI already has Aladdin visualization, and the tool presented in this notebook is complementary.

The utility functions defined in `footprint_utils.py` use one of the following APT special outputs:

* A pointings file -- this file includes the pointing information from all pointing positions in a given APT program.
* A simulator input file -- this file, originally intended to work as input for Roman's image simulation software: `romanIsim`, contains information about all exposures and the position of the `WFI_CEN` aperture, together with duration, and filter element.


On this notebook we will demonstrate how to visualize both. Additionally, we will demonstrate how to build these visualizations serially or in parallel. Note: In order to take full advantage of parallelization, the parameters (number of threads / number of processes) need to be optimized according to the machine and inputs used.


Two packages that will be used throughout the notebook are `healsparse` and `skyproj`. If you want to install these, please uncomment the cell below:

In [ ]:
!pip install healpy
!pip install healsparse
!pip install skyproj
!pip install dask[complete]

In [ ]:
import numpy as np
from footprint_utils import *
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import astropy.table

In [ ]:
import healsparse as hsp
import skyproj

By default this notebook will allow you to produce interactive plots, however, some of the plots presented here could lead to reduced performance in certain systems. In order to generate static plots, please the `interactive` variable below to `False`

In [ ]:
interactive = True
if interactive:
    %matplotlib widget
else:
    %matplotlib inline

***

# Creating and loading APT outputs

The [Astronomer's Proposal Tool (APT)](https://www.stsci.edu/scientific-community/software/astronomers-proposal-tool-apt) allows the user to design a General Astrophysics Survey with the Wide Field Instrument, and it was used to design the Roman's Core Community Surveys. Given an APT program, the user can export a variety of products using the command line. In this notebook we will focus on two products: the `pointings` file, and the `simulator input` file.

In order to export these files the user can execute the following command in a terminal:

`PATH_TO_APT_PARENT_DIRECTORY/bin/apt -export pointing,sim PATH_TO_APT_FILE/my_apt_file.apt -nogui -output PATH_TO_OUTPUT_DIRECTORY`

This command will generate a pointings file: `my_apt_file.pointing` and a simulator input file: `my_apt_file.sim.ecsv`. In the next sections we will describe what these files contain and how to read them.

# Working with `pointing` files

## Reading a pointings file

A pointings file contains information about all survey steps, mosaic patterns and region targets defined in a program. For more information about these concepts we encourage the user follow the Roman documentation ([RDox](https://roman-docs.stsci.edu)). In `footprint_utilities` we define a parsing function to interpret these pointing files. The `read_pointings_file` function reads a pointings file as produced by APT and returns two `pandas.DataFrame` objects, that below we will name `obs_all`, and `reg_all`. The first DataFrame, `obs_all` in our case, encodes the observing strategy of a single mosaic segment (see RDox for more information: https://roman-docs.stsci.edu) on each survey step. The second one, `reg_all` in our case, contains the reference pointing positions (`ra_ref, dec_ref, PA`) of the `WFI_CEN` aperture for each segment in all of the region targets defined in the APT program that was used to create the `.pointing` file.

Below, we use an example `pointing` file resulting from an early version of Roman's High-Latitude Wide-Area Survey. However, the parser function should work for any `pointing` file.

In [ ]:
obs_all, reg_all = read_pointings_file('./aux_data/roman_hlwas.pointing')

In [ ]:
obs_all

The `obs_all` DataFrame describes the mosaic patterns for a single segment in a Survey Step (given by the `Step` column). The `Tile` column gives the information about the tile number in the mosaic, and `Exposure` is the dither number of each tile. `V2`, and `V3` are referred to the original reference position in arcsec. This DataFrame also contains Ideal offset from the dither (`Dither_X, Dither_Y`), Ideal subpixel dither offset (`Subpixel_X, Subpixel_Y`), the dither distance with respect to the reference point `dDist`, ideal offset from the mosaic (or tile size if calculated from the dimensions of the aperture -- `Tile_X, Tyle_Y`, and Total ideal offset treated as being in the pointing aperture ideal frame (`Total_X, Total_Y`). For more information about the different appertures and conventions we recommend reading the [RDox](https://roman-docs.stsci.edu).

In [ ]:
reg_all

The `reg_all` DataFrame includes the `Region` name, the RA, and Dec position of the reference point of the Region, which are expressed in the `ra_ref`, `dec_ref` columns (i.e., the center of the Region itself, do not confuse this with the reference point for each segment). The `RA` of the center of the segment (in degrees), the `Dec` of the center of the segment (in degrees) and the `V2` and `V3` angles in arcsec with respect to the reference point of the region.

## Visualizing `pointing` files

In the `obs_all` and `reg_all` dataframes we have encoded all the pointing positions in the APT program used to generate the pointings file that we have read. Now we want to visualize these. 
A convenient way to represent and manipulate the information contained in these pointings file is to build `healsparse` maps (more info at: https://healsparse.readthedocs.io). These are sparse representations of `HEALPix` maps (i.e., the do not carry in memory regions of the sky that are not observed, which is convenient for high-resolution, small-area maps). The `skyproj` (more info at: https://skyproj.readthedocs.io) package has convenient function to visualize these maps.

Unfortunately, in order to visualize the observing strategy of a certain region, a certain pass, or the observations on a given filter, we need to connect the `obs_all` and `reg_all` dataframes with information that is not (currently) available on either of these objects (nor in the pointings file itself). So we need to go the the APT GUI, open the original program, and see which Survey Step corresponds to which target / filter combination. In this example we are going to focus on visualizing one (out of two) of the regions of the HLWAS Medium tier, and we are interested in visualizing the number of exposures in the F129 filter. In our particular case, going to the APT file we see that the first pass corresponds to Survey Step 95 (see screenshot below). The second pass corresponds to Survey Step 96.

<img src="./aux_data/screenshot.png" width="800" height="600"/>


In [ ]:
# We select only observations that correspond to Step 95 (first pass)
obs_filter = (obs_all['Step'] == 95) 
reg_filter = reg_all['Region'] == 'HLWAS-medium-field1\n'
print('Number of exposures per segment:', np.count_nonzero(obs_filter))
print('Number of segments:', np.count_nonzero(reg_filter))

The idea now is to perform a `for` loop that, for every segment in that region (i.e., reg_all[reg_filter]), goes over all the exposures in said segment (i.e., the observations in `obs_all[obs_filter]`.

So, a given entry in `reg_all[reg_filter]` will contain the reference position for one segment, and we will use the `v2, v3` information in `obs_all[obs_filter]` to build the full mosaic in the segment.

We show an example below and execute this serially.

In [ ]:
ra_ref0, dec_ref0, pa_ref0 = reg_all[reg_filter]['RA'].values[0], reg_all[reg_filter]['Dec'].values[0], reg_all[reg_filter]['PA'].values[0]
map_all = None
flip = -1  # In APT v.2025.4 the sign of V2, V3 are flipped with respect to the `pysiaf` convention
nside_cov = 64  # Nside for coverage map -- set it a low number if the region of the sky to be covered is large
nside_sparse = 8192  # Nside for the high-resolution map -- the larger the number the higher the resolution of the map, but the more memory is needed

for v2, v3 in zip(obs_all[obs_filter]['V2'].values, obs_all[obs_filter]['V3'].values):
    map_here = build_single_exp_map_ref(ra_ref0, dec_ref0, pa_ref0, 
                                        flip*v2, flip*v3,
                                        nside_cov, nside_sparse)
    if map_all is None:
        map_all = map_here
    else:
        map_all = hsp.sum_union([map_all, map_here])

In [ ]:
fig = plt.figure(1, figsize=(8, 6))
fig.clf()
ax = fig.add_subplot(111)
sp = skyproj.McBrydeSkyproj(ax=ax)

# We define here a custom colorbar for illustration purposes
cmap = plt.cm.Blues  # define the colormap
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0, 4, 5)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
# Note that the default is to zoom in to the range defined by the map
_ = sp.draw_hspmap(map_all, cmap=cmap, norm=norm)
# Set up the colorbar
cbar, _ = sp.draw_inset_colorbar(label='Number of exposures', height='3%', bbox_to_anchor=(-0.06, 0.01, 1, 1), loc=4, ticks=[1, 2, 3])
cbar.ax.set_xticklabels(['1', '2', '3'])
plt.show()

With large programs execute these loops serially becomes cumbersome, since each segment is independent, this is a simple-to-parallelize problem. We include an example of how to do so below.

In this case we will use `dask`, and the helper function `partial` in order to compress the arguments of
`build_single_exp_map_ref`.

In [ ]:
from dask.distributed import Client, progress, as_completed
import dask.array as da
import dask
from functools import partial
client = Client(processes=True)  # In some instances processes=False will perform better

In [ ]:
# First we create a list with all of the ra, dec, pa positions for the reference

ra_ref_all = reg_all[reg_filter]['RA'].values
dec_ref_all = reg_all[reg_filter]['Dec'].values
pa_ref_all = reg_all[reg_filter]['PA'].values

# Collect all v2, v3 from our observations

v2_all = obs_all[obs_filter]['V2'].values
v3_all = obs_all[obs_filter]['V3'].values

# Repeat them and construct an array with nsegments x nexp

ra_ref_all = ra_ref_all[:, None] * np.ones(v3_all.shape[0])
dec_ref_all = dec_ref_all[:, None] * np.ones(v3_all.shape[0])
pa_ref_all = pa_ref_all[:, None] * np.ones(v3_all.shape[0])

v2_all = v2_all[None, :] * np.ones(ra_ref_all.shape[0])[:, None]
v3_all = v3_all[None, :] * np.ones(ra_ref_all.shape[0])[:, None]

# Flatten to pass to our multiprocessing function

ra_ref_all = da.from_array(ra_ref_all.flatten())
dec_ref_all = da.from_array(dec_ref_all.flatten())
pa_ref_all = da.from_array(pa_ref_all.flatten())
v2_all = da.from_array(v2_all.flatten())
v3_all = da.from_array(v3_all.flatten())


In [ ]:
# Create the mapping task for building exposures for each single exposure
map_here = client.map(partial(build_single_exp_map_ref, nside_sparse=nside_sparse, nside_cov=nside_cov), ra_ref_all,
                       dec_ref_all, pa_ref_all, v2_all, v3_all)

In [ ]:
# Add all together
total = client.map(hsp.sum_union, [map_here])

In [ ]:
# Create progress bar and compute
progress(total)

In [ ]:
# Create a view of the result in a variable (or compute and bring to memory if it has not been computed on the previous step)
map_new = total[0].result()

In [ ]:
fig = plt.figure(2, figsize=(8, 6))
fig.clf()
ax = fig.add_subplot(111)
sp = skyproj.McBrydeSkyproj(ax=ax)

# We define here a custom colorbar for illustration purposes
cmap = plt.cm.Blues  # define the colormap
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0, 4, 5)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
# Note that the default is to zoom in to the range defined by the map
_ = sp.draw_hspmap(map_new, cmap=cmap, norm=norm)
# Set up the colorbar
cbar, _ = sp.draw_inset_colorbar(label='Number of exposures', height='3%', bbox_to_anchor=(-0.06, -0.1, 1, 1), loc=1, ticks=[1, 2, 3])
cbar.ax.set_xticklabels(['1', '2', '3'])
plt.show()

One more example, now with two passes:

In [ ]:
obs_filter = (obs_all['Step'] == 95) | (obs_all['Step'] == 96)
reg_filter = reg_all['Region'] == 'HLWAS-medium-field1\n'
print('Number of exposures per segment:', np.count_nonzero(obs_filter))
print('Number of segments:', np.count_nonzero(reg_filter))

# First we create a list with all of the ra, dec, pa positions for the reference

ra_ref_all = reg_all[reg_filter]['RA'].values
dec_ref_all = reg_all[reg_filter]['Dec'].values
pa_ref_all = reg_all[reg_filter]['PA'].values

# Collect all v2, v3 from our observations

v2_all = obs_all[obs_filter]['V2'].values
v3_all = obs_all[obs_filter]['V3'].values

# Repeat them and construct an array with nsegments x nexp

ra_ref_all = ra_ref_all[:, None] * np.ones(v3_all.shape[0])
dec_ref_all = dec_ref_all[:, None] * np.ones(v3_all.shape[0])
pa_ref_all = pa_ref_all[:, None] * np.concatenate([np.ones(v3_all.shape[0]//2), np.ones(v3_all.shape[0]//2)*227/60])  # Trick to force the second pass to PA=227 -- feature from APT 2025.4 exporter

v2_all = v2_all[None, :] * np.ones(ra_ref_all.shape[0])[:, None]
v3_all = v3_all[None, :] * np.ones(ra_ref_all.shape[0])[:, None]


# Flatten to pass to our multiprocessing function

ra_ref_all = da.from_array(ra_ref_all.flatten())
dec_ref_all = da.from_array(dec_ref_all.flatten())
pa_ref_all = da.from_array(pa_ref_all.flatten())
v2_all = da.from_array(v2_all.flatten())
v3_all = da.from_array(v3_all.flatten())

In [ ]:
# We reuse the client
map_here = client.map(partial(build_single_exp_map_ref, nside_sparse=nside_sparse, nside_cov=nside_cov), ra_ref_all,
                       dec_ref_all, pa_ref_all, v2_all, v3_all)

In [ ]:
total = client.map(hsp.sum_union, [map_here])

In [ ]:
progress(total)

In [ ]:
map_new = total[0].result()

In [ ]:
fig = plt.figure(3, figsize=(8, 6))
fig.clf()
ax = fig.add_subplot(111)
sp = skyproj.McBrydeSkyproj(ax=ax)

# We define here a custom colorbar for illustration purposes
cmap = plt.cm.Blues  # define the colormap
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0, 7, 8)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
# Note that the default is to zoom in to the range defined by the map
_ = sp.draw_hspmap(map_new, cmap=cmap, norm=norm)
# Set up the colorbar
cbar, _ = sp.draw_inset_colorbar(label='Number of exposures', height='3%', bbox_to_anchor=(-0.06, -0.1, 1, 1), loc=1, ticks=[1, 3, 6])
cbar.ax.set_xticklabels(['1', '3', '6'])
plt.show()

In [ ]:
client.close()

The non uniformities are caused by the fact that the region tiling has been defined using the original PA angle (60 degrees). For future versions this will be updated.

# Working with simulator input files

## Reading a simulator input file `sim.ecsv`

The simulator input file `*.sim.ecsv` is an output from APT intended to be used as input in `romanisim` to generate simulated images. It contains information about the position `RA, Dec` of the `WFI_CEN` aperture for eaach exposure in the program, together with information about the filter element used, and the duration in seconds (Note: duration is not exposure time, but exposure time + potential slew times / estimated overheads). These files are in `ecsv` format so they can easily be read by `pandas` or `astropy` (recommended for automatic parsing).

In [ ]:
sim_table = astropy.table.Table.read('./aux_data/roman_hlwas.sim.ecsv')

In [ ]:
sim_table

In this case we opted to use `astropy` and got an `astropy.table.Table` object that contains the `RA, DEC, PA` of the `WFI_CEN` aperture. The filter element is displayed in the `BANDAPASS` column. The `MA_TABLE_NUMBER` column indicates the identification number of the MultiAccum table. The `DURATION` column indicates the duration of the exposure including overheads (slews + readout, etc). `PLAN` is the plan number. `PASS` corresponds to `Survey Step` in APT. `SEGMENT` corresponds to the segment number. `OBSERVATION` corresponds to the observation number in a survey step (i.e., the pass number in a survey step and it matches `Observation` in the `obs_all` DataFrame). `VISIT` corresponds to the `Tile` number in the mosaic segment (in the `obs_all` DataFrame). Finally, `EXPOSURE` is the exposure (dither) number for a tile.

So, now, we want to visualize all the exposures in the `F158` filter, and check the total exposure time. So we will take the `sim_table` Table and select all exposures with `BANDPASS == 'F158`. Then we will accumulate the exposure time. The exposures with `MA_TABLE_NUMBER == 5` have an exposure time of 295 seconds, whereas the exposures with `MA_TABLE_NUMBER == 6` have an exposure time of 107 seconds.

In [ ]:
table_mask = sim_table['BANDPASS'] == 'F158'
sub_table = sim_table[table_mask]
exp_time = np.ones(np.count_nonzero(table_mask))
exp_time[sub_table['MA_TABLE_NUMBER'] == 6] = 295
exp_time[sub_table['MA_TABLE_NUMBER'] == 5] = 107
ra_sim = sub_table['RA'].data
dec_sim = sub_table['DEC'].data
pa_sim = sub_table['PA'].data

print('Number of selected exposures', len(sub_table))

In this case, instead of a reference position and the relative `V2`, `V3` angles, we directly have the `RA`, `DEC`, and `PA` of the `WFI_CEN` aperture so we can used these directly via `build_single_exp_map_cen`. Let's start with a single segment within the HLWAS medium field (Survey Step 97).

In [ ]:
mask_here = (sub_table['PASS'] == 97) & (sub_table['SEGMENT'] == 1)
print('Going to use', np.count_nonzero(mask_here), 'exposures')
map_result = None
for ra, dec, pa, et in zip(ra_sim[mask_here], dec_sim[mask_here], pa_sim[mask_here], exp_time[mask_here]):
    map_here = build_single_exp_map_cen(ra, dec, pa, et, nside_cov=nside_cov, nside_sparse=nside_sparse)
    if map_result is None:
        map_result = map_here
    else:
        map_result = hsp.sum_union([map_result, map_here])

In [ ]:
fig = plt.figure(4, figsize=(8, 6))
fig.clf()
ax = fig.add_subplot(111)
sp = skyproj.McBrydeSkyproj(ax=ax)

# We define here a custom colorbar for illustration purposes
cmap = plt.cm.Blues  # define the colormap
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0, 400, 100)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
# Note that the default is to zoom in to the range defined by the map
_ = sp.draw_hspmap(map_result, cmap=cmap, norm=norm)
# Set up the colorbar
cbar, _ = sp.draw_inset_colorbar(label='Total exposure time (s)', height='3%', bbox_to_anchor=(-0.06, -0.1, 1, 1), loc=1, ticks=[107, 214, 321])
cbar.ax.set_xticklabels(['107', '214', '321'])
plt.show()

Now, let's do all of the exposures using the F158 filter element.

In [ ]:
niter = 15
ra_sim = sub_table['RA'].data
dec_sim = sub_table['DEC'].data
pa_sim = sub_table['PA'].data

In [ ]:
# Re-open the dask client
client = Client(processes=True)

In [ ]:
map_all = []
for i in range(niter):
    print('Iteration', i)
    imin = (i*len(ra_sim))//niter
    imax = ((i+1)*len(ra_sim))//niter
    ra_subset = ra_sim[imin:imax]
    dec_subset = dec_sim[imin:imax]
    pa_subset = pa_sim[imin:imax]
    exp_subset = exp_time[imin:imax]
    map_here = client.map(partial(build_single_exp_map_cen, nside_sparse=nside_sparse, nside_cov=nside_cov),
                          ra_subset, dec_subset, pa_subset, exp_subset)
    map_partial = client.map(hsp.sum_union, [map_here])
    res_here = map_partial[0].result()
    map_all.append(res_here)


In [ ]:
client.close()

In [ ]:
map_f158 = hsp.sum_union(map_all)

In [ ]:
fig = plt.figure(5, figsize=(10, 6))
fig.clf()
ax = fig.add_subplot(111)
sp = skyproj.McBrydeSkyproj(ax=ax)

# We define here a custom colorbar for illustration purposes
cmap = plt.cm.Blues  # define the colormap
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0, 3000, 200)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
# Note that the default is to zoom in to the range defined by the map
_ = sp.draw_hspmap(map_f158, cmap=cmap, norm=norm)
# Set up the colorbar
cbar, _ = sp.draw_inset_colorbar(label='Total exposure time (s)', height='100%', width='1%', orientation='vertical',
                                 bbox_to_anchor=(+0.06, -0.1, 1, 1), loc=1)
plt.show()

Note that the resulting `healsparse` maps can be saved to disk using the `write` method.

Disclaimer: Depending on the map resolution (controlled by the `nside` parameter) it can generate a very large file.

In [ ]:
# map_f158.write('./data/map_f158_all.hsp')  # Uncomment if you want to write the map to disk. With the default parameters in this notebook it should use 45 MB.

## About this Notebook

**Author(s):** Javier Sánchez <br>
**Keyword(s):** Tutorial, visualization, survey footprints <br>
**Last Updated:** Oct 2025 <br>
**Next Review:** 
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 